In [1]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Sheet Music Analysis with Gemini

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/document-processing/sheet_music.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fuse-cases%2Fdocument-processing%2Fsheet_music.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>       
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/document-processing/sheet_music.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/use-cases/document-processing/sheet_music.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>


| | |
|-|-|
|Author(s) | [Holt Skinner](https://github.com/holtskinner) |

## Overview

[Sheet Music](https://en.wikipedia.org/wiki/Sheet_music) is the primary form of music notation used by composers and performers across the world. These pages contain information about the lyrics, pitches, rhythms, composer, text author, composition date, among others.

This notebook illustrates using Gemini to extract this metadata from sheet music PDFs.

These prompts and documents were demonstrated in the [Google Cloud Next 2024 session "What's next with Gemini: Driving business impact with multimodal use cases"](https://www.youtube.com/watch?v=DqH1R9Pk5RI).


## Getting Started


### Install Vertex AI SDK for Python

In [2]:
%pip install --upgrade --user -q google-cloud-aiplatform PyPDF2

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [3]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment. This step is not required if you are using [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench).


In [1]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [2]:
# Define project information
PROJECT_ID = "qwiklabs-gcp-04-93d7352957fa"  # @param {type:"string"}
LOCATION = "europe-west1"  # @param {type:"string"}

# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [3]:
# Please like share & subscribe to Techcps
# YouTube https://www.youtube.com/@techcps

print("Please like share & subscribe to Techcps https://www.youtube.com/@techcps")

Please like share & subscribe to Techcps https://www.youtube.com/@techcps


### Import libraries


In [4]:
import json

from IPython.display import Markdown, display
import PyPDF2
from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
)

### Load the Gemini 1.5 Flash model

Gemini 1.5 Flash (`gemini-1.5-flash`) is a multimodal model that supports multimodal prompts. You can include text, image(s), PDFs, audio, and video in your prompt requests and get text or code responses.

In [5]:
generation_config = GenerationConfig(temperature=1.0, max_output_tokens=8192)
safety_settings = {
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH
}
model = GenerativeModel(
    model_name="gemini-1.5-flash",
    system_instruction="You are an expert in musicology and music history.",
    generation_config=generation_config,
    safety_settings=safety_settings,
)

## Extract Structured Metadata from Sheet Music PDF

For this example, we will be using the popular classical music book [24 Italian Songs and Arias of the 17th and 18th Centuries](https://imslp.org/wiki/24_Italian_Songs_and_Arias_of_the_17th_and_18th_Centuries_(Various)), and extracting metadata about each song in the book.

In [6]:
sheet_music_pdf_uri = "gs://github-repo/use-cases/sheet-music/24ItalianSongs.pdf"

sheet_music_extraction_prompt = """The following document is a book of sheet music. Your task is to output structured metadata about every piece of music in the document. Correct any mistakes that are in the document and fill in missing information when not present in the document.

Include the following details:

Title
Composer with lifetime
Tempo Marking
Composition Year
A description of the piece
"""

# Load file directly from Google Cloud Storage
file_part = Part.from_uri(
    uri=sheet_music_pdf_uri,
    mime_type="application/pdf",
)

# Send to Gemini
response = model.generate_content([sheet_music_extraction_prompt, file_part])

# Display results
display(Markdown(response.text))

## Twenty-Four Italian Songs and Arias

**1.  Per la gloria d'adorarvi**
- **Composer:** Giovanni Battista Bononcini (1672-1750)
- **Tempo Marking:** Andante = 80
- **Composition Year:** Not Present
- **Description:** An aria from the opera "Griselda"

**2. Amarilli, mia bella**
- **Composer:** Giulio Caccini (1546-1618)
- **Tempo Marking:** Moderato affettuoso = 66
- **Composition Year:** Not Present
- **Description:** A Madrigal

**3. Alma del core**
- **Composer:** Antonio Caldara (1670-1736)
- **Tempo Marking:** Tempo di Minuetto
- **Composition Year:** Not Present
- **Description:** An aria.

**4. Comme raggio di sol**
- **Composer:** Antonio Caldara (1670-1736)
- **Tempo Marking:** Sostenuto = 46
- **Composition Year:** Not Present
- **Description:** An aria.

**5. Sebben, crudele**
- **Composer:** Antonio Caldara (1670-1736)
- **Tempo Marking:** Allegretto grazioso = 84
- **Composition Year:** Not Present
- **Description:** A canzonetta.

**6. Vittoria, mio core!**
- **Composer:** Giacomo Carissimi (1605-1674)
- **Tempo Marking:** Allegro con brio = 168
- **Composition Year:** Not Present
- **Description:** A cantata.

**7. Danza, danza, fanciulla gentile**
- **Composer:** Francesco Durante (1684-1755)
- **Tempo Marking:** Allegro con spirito = 138
- **Composition Year:** Not Present
- **Description:** An arietta.

**8. Vergin, tutto amor**
- **Composer:** Francesco Durante (1684-1755)
- **Tempo Marking:** Largo religioso = 40
- **Composition Year:** Not Present
- **Description:** A prayer.

**9. Caro mio ben**
- **Composer:** Giuseppe Giordani (Giordanello) (1744-1798)
- **Tempo Marking:** Larghetto = 60
- **Composition Year:** Not Present
- **Description:** An arietta.

**10. Che fiero costume**
- **Composer:** Giovanni Legrenzi (1626-1690)
- **Tempo Marking:** Allegretto con moto = 56
- **Composition Year:** Not Present
- **Description:** An arietta.

**11. Pur dicesti, o bocca bella**
- **Composer:** Antonio Lotti (1667-1740)
- **Tempo Marking:** Allegretto grazioso = 69
- **Composition Year:** Not Present
- **Description:** An arietta.

**12. Lasciatemi morire!**
- **Composer:** Claudio Monteverdi (1567-1643)
- **Tempo Marking:** Lento = 58
- **Composition Year:** Not Present
- **Description:** An aria from the opera "Ariana".

**13. Nel cor più non mi sento**
- **Composer:** Giovanni Paisiello (1740-1816)
- **Tempo Marking:** Andantino = 58
- **Composition Year:** Not Present
- **Description:** An arietta.

**14. Se tu m'ami, se sospiri**
- **Composer:** Giovanni Battista Pergolesi (1710-1736)
- **Tempo Marking:** Andantino = 58
- **Composition Year:** Not Present
- **Description:** An arietta.

**15. Nina**
- **Composer:** Legrenzio Vincenzo Ciampi (1719-?)
- **Tempo Marking:** Andantino
- **Composition Year:** Not Present
- **Description:** A canzonetta. 

**16. Già il sole dal Gange**
- **Composer:** Alessandro Scarlatti (1659-1725)
- **Tempo Marking:** Allegro giusto = 126
- **Composition Year:** Not Present
- **Description:** A canzonetta.

**17. O cessate di piagarmi**
- **Composer:** Alessandro Scarlatti (1659-1725)
- **Tempo Marking:** Andante con moto = 80
- **Composition Year:** Not Present
- **Description:** An arietta.

**18. Se Florindo è fedele**
- **Composer:** Alessandro Scarlatti (1659-1725)
- **Tempo Marking:** Allegretto grazioso, moderato assai = 132
- **Composition Year:** Not Present
- **Description:** An arietta.

**19. O del mio dolce ardor**
- **Composer:** Christoph Willibald von Gluck (1714-1787)
- **Tempo Marking:** Moderato = 46
- **Composition Year:** Not Present
- **Description:** An aria.

**20. Pietà, Signore!**
- **Composer:** Alessandro Stradella (1645?-1682)
- **Tempo Marking:** Andantino
- **Composition Year:** Not Present
- **Description:** An aria.

**21. Tu lo sai**
- **Composer:** Giuseppe Torelli (1650-1703)
- **Tempo Marking:** Andantino
- **Composition Year:** Not Present
- **Description:** An aria. 


You can see that Gemini extracted all of the relevant fields from the document.

### Song Identification with Audio

Now, let's try something more challenging, identifying a song being performed based on the sheet music. We have an audio clip of Holt Skinner performing one of the songs in the book, and we will ask Gemini to identify it based on the sheet music PDF.

In [7]:
song_identification_prompt = """Based on the sheet music PDF, what song is in the audio clip? Explain how you made the decision."""

# Load PDF file
pdf_part = Part.from_uri(
    uri=sheet_music_pdf_uri,
    mime_type="application/pdf",
)

audio_part = Part.from_uri(
    uri="gs://github-repo/use-cases/sheet-music/24ItalianClip.mp3",
    mime_type="audio/mpeg",
)

# Send to Gemini
response = model.generate_content([pdf_part, audio_part, song_identification_prompt])

# Display results
display(Markdown(response.text))

The audio clip is from the song "Sebben, crudele". It is the only song from the sheet music PDF that contains a section with a repeated word "sempre". 


### Edit PDF Metadata

Next, we'll use the output from Gemini to edit the metadata of a PDF containing one song, which can make it easier to organize this file in sheet music applications.

We'll adjust the prompt slightly and set the [`response_mime_type`](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/gemini#:~:text=in%20the%20list.-,responseMimeType,-(Preview)) to get the response in JSON format.

In [8]:
sheet_music_pdf_uri = "gs://github-repo/use-cases/sheet-music/SebbenCrudele.pdf"

sheet_music_extraction_prompt = """The following document is a piece of sheet music. Your task is to output structured metadata about the piece of music in the document. Correct any mistakes that are in the document and fill in missing information when not present in the document.

Output the data in the following JSON format:

{
    "/Title": "Title of the piece",
    "/Author": "Composer(s) of the piece",
    "/Subject": "Music Genre(s) in a comma separated list",
}

"""

# Load file directly from Google Cloud Storage
file_part = Part.from_uri(
    uri=sheet_music_pdf_uri,
    mime_type="application/pdf",
)

generation_config = GenerationConfig(
    temperature=1, response_mime_type="application/json"
)

# Send to Gemini
response = model.generate_content(
    [sheet_music_extraction_prompt, file_part], generation_config=generation_config
)

# Display results
display(Markdown(response.text))

new_metadata = json.loads(response.text)

{"/Title": "Sebben, crudele", "/Author": "Antonio Caldara", "/Subject": "Canzonetta, Vocal"}

Next, we'll download the PDF from the GCS Bucket and edit the metadata using the [`PyPDF2`](https://pypdf2.readthedocs.io/en/3.x/) library.

In [9]:
! gcloud storage cp {sheet_music_pdf_uri} .

Copying gs://github-repo/use-cases/sheet-music/SebbenCrudele.pdf to file://./SebbenCrudele.pdf
  Completed files 1/1 | 586.7kiB/586.7kiB                                      

Average throughput: 32.7MiB/s


In [10]:
def edit_pdf_metadata(file_path: str, new_metadata: dict) -> None:
    """Edits metadata of a PDF file.

    Args:
        file_path (str): Path to the PDF file.
        new_metadata (dict): Dictionary containing the new metadata fields and values.
            Example: {'/Author': 'John Doe', '/Title': 'My Report'}
    """

    with open(file_path, "rb") as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        pdf_writer = PyPDF2.PdfWriter()

        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            pdf_writer.add_page(page)

        pdf_writer.add_metadata(new_metadata)

        with open(file_path, "wb") as out_file:
            pdf_writer.write(out_file)


edit_pdf_metadata("SebbenCrudele.pdf", new_metadata)